In [1]:
import pandas as pd  


In [2]:
outgoing_df = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")

In [3]:
outgoing_df.groupby("base").count()

,link,type
base,,
50 Cent,29,29
Aaron Lebedeff,29,29
Act Up,28,28
Al Harewood,30,30
Alexander Jackson Davis,26,26
...,...,...
Werner Janssen (Komponist),29,29
Westminster Kennel Club Dog Show,28,28
William Beach Lawrence,29,29


In [4]:
views_df = pd.read_csv("../Data/Köln/views.csv")
views_df[(views_df["views"] > 100) & (views_df["link"].isin(outgoing_df["base"]))]

,link,date,views
49656,Krieg von Castellammare,2023092500,132
53038,Bonnie Bedelia,2023070600,112
53055,Bonnie Bedelia,2023072300,192
53058,Bonnie Bedelia,2023072600,185
53062,Bonnie Bedelia,2023073000,162
...,...,...,...
180048,John Barrett (Friseur),2023082100,214
180049,John Barrett (Friseur),2023082200,104
182507,Internationales Hot-Dog-Wettessen,2023070400,271
182508,Internationales Hot-Dog-Wettessen,2023070500,339


Fill all empty dates in view_df with 0

In [5]:
all_dates = views_df[views_df["link"] == "50 Cent"]["date"]
# for link in views_df["link"]:
#   merged = 
#filled_views_df = list(map(lambda link: pd.merge(views_df[views_df["link"] == link], all_dates, how="right", on="date").fillna({"link": link, "views": 0}), list(views_df["link"])))

In [11]:
filled_df = pd.read_csv("../Data/Köln/filled_views.csv")
filled_df

,link,date,views
0,FIFA-Weltfußballer des Jahres,2023070100,270.0
1,FIFA-Weltfußballer des Jahres,2023070200,871.0
2,FIFA-Weltfußballer des Jahres,2023070300,232.0
3,FIFA-Weltfußballer des Jahres,2023070400,215.0
4,FIFA-Weltfußballer des Jahres,2023070500,198.0
...,...,...,...
203572,COVID-19-Pandemie,2023092700,913.0
203573,COVID-19-Pandemie,2023092800,876.0
203574,COVID-19-Pandemie,2023092900,811.0
203575,COVID-19-Pandemie,2023093000,715.0


In [7]:
links = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")
links

,base,link,type
0,Charlie Barnett (Komiker),1954,outgoing
1,Charlie Barnett (Komiker),Komiker,outgoing
2,Charlie Barnett (Komiker),Afroamerikaner,outgoing
3,Charlie Barnett (Komiker),New York City,outgoing
4,Charlie Barnett (Komiker),West Virginia,outgoing
...,...,...,...
2545,Michael Blake (Musiker),Lounge Lizards,outgoing
2546,Michael Blake (Musiker),Billy Martin (Schlagzeuger),outgoing
2547,Michael Blake (Musiker),David Tronzo,outgoing
2548,Michael Blake (Musiker),Klarinette,outgoing


In [12]:
combination = filled_df.merge(links,on="link",how="right")
ml_output = []
ml_link = []
for base in links["base"].sort_values().unique():
  ml_output += filled_df[filled_df["link"] == base].sort_values("date")["views"].to_list()
  ml_link += filled_df[filled_df["link"] == base]["link"].to_list()
len(ml_link)

9300

In [14]:
res = []
for idx in range(0, len(ml_output)) :
    if ml_output[idx] > 100:
        res.append(idx)

In [15]:
combination = combination.sort_values(["base", "date"], ignore_index=True)
ml_input = []
for base in combination["base"].unique():
  for date in combination["date"].unique():
    ml_input.append(combination[(combination["base"] == base) & (combination["date"] == date)])


In [16]:
ml_input[2]["date"]

timestamp = 2023070100
datetime = pd.to_datetime(timestamp, format='%Y%m%d%H')

print(datetime)

2023-07-01 00:00:00


In [41]:
ml_input_value = [e["views"].tolist() for e in ml_input]

In [40]:
ml_input[29]

,link,date,views,base,type
841,New York City,2023073000,1921.0,50 Cent,outgoing
842,Run-D.M.C.,2023073000,150.0,50 Cent,outgoing
843,Recording Industry Association of America,2023073000,33.0,50 Cent,outgoing
844,1975,2023073000,51.0,50 Cent,outgoing
845,Billboard Hot 100,2023073000,115.0,50 Cent,outgoing
846,Goldene Schallplatte,2023073000,174.0,50 Cent,outgoing
847,Vereinigte Staaten,2023073000,4932.0,50 Cent,outgoing
848,Deutsche Singlecharts,2023073000,246.0,50 Cent,outgoing
849,Präsidentschaftswahl in den Vereinigten Staate...,2023073000,248.0,50 Cent,outgoing
850,Candy Shop,2023073000,26.0,50 Cent,outgoing


In [42]:
ml_input_value[29]

[1921.0,
 150.0,
 33.0,
 51.0,
 115.0,
 174.0,
 4932.0,
 246.0,
 248.0,
 26.0,
 6.0,
 1160.0,
 752.0,
 128.0,
 41.0,
 81.0,
 18.0,
 78.0,
 20.0,
 0.0,
 9.0,
 20.0,
 2817.0,
 69.0,
 35.0,
 996.0,
 69.0,
 203.0,
 69.0]

In [45]:
import numpy as np


for i in range(len(ml_input_value)):
  if len(ml_input_value[i]) < 30:
    zero_list = list(np.zeros(30- len(ml_input_value[i])))
    ml_input_value[i] = ml_input_value[i] + zero_list

ml_input_value[0]

[1556.0,
 174.0,
 22.0,
 44.0,
 108.0,
 221.0,
 4378.0,
 228.0,
 186.0,
 37.0,
 8.0,
 1003.0,
 522.0,
 101.0,
 22.0,
 133.0,
 40.0,
 57.0,
 36.0,
 0.0,
 15.0,
 61.0,
 1718.0,
 89.0,
 53.0,
 989.0,
 26.0,
 238.0,
 84.0,
 0.0]

In [59]:
for_csv = {"input":ml_input_value,"output":ml_output}
save_as_file = pd.DataFrame(for_csv)
save_as_file["input"]
# save_as_file.to_csv("../Data/Köln/ml_training_data.csv", index=False)

0       [1556.0, 174.0, 22.0, 44.0, 108.0, 221.0, 4378...
1       [1981.0, 159.0, 28.0, 70.0, 115.0, 179.0, 4792...
2       [1800.0, 96.0, 19.0, 67.0, 79.0, 174.0, 4872.0...
3       [1715.0, 119.0, 20.0, 76.0, 88.0, 182.0, 5710....
4       [1776.0, 114.0, 39.0, 75.0, 107.0, 177.0, 4933...
                              ...                        
9295    [1954.0, 4.0, 2267.0, 167.0, 2.0, 2.0, 835.0, ...
9296    [1861.0, 3.0, 2135.0, 155.0, 3.0, 2.0, 789.0, ...
9297    [1993.0, 3.0, 2068.0, 151.0, 2.0, 1.0, 812.0, ...
9298    [2257.0, 5.0, 1934.0, 85.0, 1.0, 1.0, 787.0, 6...
9299    [2182.0, 9.0, 2334.0, 118.0, 6.0, 0.0, 1004.0,...
Name: input, Length: 9300, dtype: object

In [46]:
#clf.predict(ml_input_value[7900:8000])
np.shape(ml_input_value)

(9300, 30)

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = scaler.fit_transform(ml_input_value)
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeRegressor(random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 13.   0.   0. ... 218.   1.  28.]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 0.0, 27.0, 1

In [48]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics


scaler = StandardScaler()

X = scaler.fit_transform(ml_input_value)
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeRegressor(random_state=0)

model.fit(X_train, y_train)
sel = SelectFromModel(model)
X_trans = sel.fit_transform(X_train, y_train)

model.fit(X_trans, y_train)

X_test_trans = sel.transform(X_test)


# make predictions
y_pred = model.predict(X_test_trans)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 11.   0.   0. ... 205.   0.  29.]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 0.0, 27.0, 1

In [49]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=100, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [1.1040e+01 1.2000e-01 1.1600e+00 ... 1.9398e+02 1.1600e+00 4.5560e+01]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.

In [50]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=200, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [1.11300e+01 1.90000e-01 1.11500e+00 ... 1.97595e+02 1.08000e+00
 4.48550e+01]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 

In [157]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=100, random_state=0)

model.fit(X_train, y_train)
sel = SelectFromModel(model)
X_trans = sel.fit_transform(X_train, y_train)

model.fit(X_trans, y_train)

X_test_trans = sel.transform(X_test)

# make predictions
y_pred = model.predict(X_test_trans)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 11.41   0.2    0.93 ... 198.33   1.96  40.9 ]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=200, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [190]:
ml_input_value = [e["views"].tolist() + list(np.empty(30- len(e["views"].tolist()))) + [float(pd.to_datetime(e.iloc[0]["date"], format='%Y%m%d%H').dayofweek)] for e in ml_input]

In [199]:
max(ml_input_value)

[13618.0,
 10.0,
 73.0,
 1780.0,
 1.0,
 115.0,
 1125.0,
 237.0,
 4323.0,
 131.0,
 3.0,
 7.0,
 39.0,
 2052.0,
 5.0,
 38.0,
 404.0,
 50.0,
 4.0,
 418.0,
 229.0,
 2.0,
 4.0,
 126.0,
 13.0,
 0.0,
 0.0,
 218.0,
 0.0,
 29.0,
 2.0]

In [198]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeRegressor(random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

c:\Users\SeipelMa\Code\vscode-workspace\DataScience\DataflowAnalysis\.venv\Lib\site-packages\sklearn\utils\_array_api.py:521: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: Input X contains infinity or a value too large for dtype('float32').